In [ ]:
# The only thing that differs really, the model part itself

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall

# Define the ResNet base model with ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)

x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)

x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers initially for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Optionally, unfreeze the last few layers for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Compile the model with an adaptive optimizer and additional metrics
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)

# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)

checkpoint = ModelCheckpoint(
    filepath='best_model_resnet.keras',  # Path to save the model
    monitor='val_loss',                 # Metric to monitor
    mode='max',                         # 'min' for loss, 'max' for accuracy
    save_best_only=True,                # Save only the best weights
    verbose=1                           # Display a message when a model is saved
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    validation_data=val_generator,  # Pass validation generator directly
    validation_steps=val_generator.samples // val_generator.batch_size
)
